<div class="alert alert-block alert-info">
<b>Achtung:</b> Der hier angezeigt Code kann nicht direkt im Exercise Server ausgeführt werden. Der Code fuktioniert nur im <a href="http://localhost:8080/lab">Jupyter Lab der Development Environment</a>.</div>

# Ein einfaches ML Modell

In dieser Übung wird als Grundlage für die nachfolgenden Übungen ein einfaches Machine Learning Modell trainiert. Der Vorgang simuliert die Arbeit der *Data Scientisten*, welche Daten analysieren und Modelle bauen.

# Vorbereitung

## Notebook für den Code erstellen

Erstelle als erstes in der Development Environment ein neues, leeres Jupyter Notebook, in welchem du die nachfolgenden Aufgaben lösen kannst. Nütze auch die Markdown-Funkionalität, um deine Gedanken zu notieren und um zu dokumentieren!

Nenne das Notebook `01-Simple_Model.ipynb`

## Trainingsdaten downloaden

Zur Vorbereitung müssen Trainingsdaten im Object Store platziert werden. Führe dazu die folgenden Schritte durch, um das File downzuloaden.

Lade [dieses Zipfile](https://www.kaggle.com/api/v1/datasets/download/prishasawhney/mushroom-dataset?datasetVersionNumber=1) herunter, entpacke es, und lies daraus das file 'mushroom_cleaned.csv' in einen pandas dataframe ein.

Hint: Du kannst dies mit [requests](https://docs.python-requests.org/en/latest/index.html), [zipfile](https://docs.python.org/3/library/zipfile.html) und [io](https://docs.python.org/3/library/io.html) direkt machen. Oder Du holst das File mit deinem Browser und uploadest das CSV ins Jupyter.

**Wichtig**: Lies das File direkt mit read_csv(), gib dabei keine Optionen an.

In [ ]:
import requests, zipfile, io
import pandas as pd

url = 'https://www.kaggle.com/api/v1/datasets/download/prishasawhney/mushroom-dataset?datasetVersionNumber=1'
r = requests.get(url)

with zipfile.ZipFile(io.BytesIO(r.content)) as z:
    with z.open('mushroom_cleaned.csv') as f:
        df = pd.read_csv(f)

----

Erstelle nun einen Bucket mit Name `traindata` im objectstore.

In [ ]:
import s3fs

s3 = s3fs.S3FileSystem()
s3.mkdir("traindata")

----

Speichere schliesslich das csv File als Parquet unter dem Namen 'traindata/train_raw.parquet'.

Hint: Wenn du dies direkt mit pandas machst, musst du das Argument `storage_options={"anon": False}` angeben.

In [ ]:
df.to_parquet('s3://traindata/train_raw.parquet', storage_options={"anon": False})

----

# Übungen

## Das Mushroom Dataset

Das Mushroom Dataset, welches wir für einige Übungen verwenden, ist gedacht, um Pilze anhand verschiedener Merkmale in die beiden Klassen *essbar* und *giftig* einzuteilen. Es kommt in tabellarischer Form daher mit 8 Features und einem Label.

Das originale Datenset kommt aus dem [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset). Dort gibt eine Beschreibung, was die Spalten bedeuten. Für uns ist dies heute jedoch sekundär.

Die Version, welche auf Kaggle bereitgestellt wurde, ist gesäubert und enthält weniger Spalten als das Original. 

Das Label enthält zwei Werte, 0 für *essbar* und 1 für *giftig*.

## Trainingsdaten aus Object Store lesen

Verwende Pandas, um das gerade gespeicherte Parquet-File in einen Dataframe zu lesen (nach dem Ausführen von obigem Code wäre das natürlich nicht notwendig, da der DataFrame schon im Memory ist. Wir tun es aber trotzdem).

In [ ]:
import pandas as pd
df = pd.read_parquet('s3://traindata/train_raw.parquet', storage_options={"anon": False})

----

## Exploratory Data Analysis

Schaue Dir nun den DataFrame an, um ein Gefühl für die Daten zu bekommen, zum Beispiel
 * Erste paar Zeilen
 * Anzahl Spalten und Zeilen
 * Datentypen
 * Anzahl unterschiedliche Werte pro Spalte
 * NaNs
 * Mögliche Class Imbalance

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
df.isna().any().any()

In [ ]:
df['class'].value_counts()

----

Natürlich wäre hier noch viel mehr möglich, wir kürzen aber ab, da dies ein Kurs für MLOps und nicht für Modelling ist.

## Feature Engineering

Es ist nicht notwendig, neue Features zu engineeren. Wenn du möchtest, kannst du dies natürlich tun.

Wir bauen  zwei Listen `categoricals` und `numericals`, welche je die Spaltennamen der kategorischen und der numerischen Spalten enthalten.

In [ ]:
categoricals = ['cap-shape', 'gill-attachment', 'gill-color', 'stem-color']
numericals = [c for c in df.drop('class', axis='columns').columns if c not in categoricals]

----

Konvertiere alle Spalten in den Datentyp `float`. Dies wird später helfen, wenn wir mit MLFlow arbeiten, denn MLFlow bekundet Mühe mit Spalten der Typen `int` oder `categorical`.

In [ ]:
for col in df.columns:
    df[col] = df[col].astype('float')

----

## Finales Test Set vorbereiten

Mit der Funktion [train_test_split von sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) teilen wir das Datenset auf in Trainings- und Testset `X_train`, `X_test`, `y_train`, `y_test` (`random_state=42`, ansonsten default-Parameter belassen).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis='columns'), df['class'], random_state=42)

----

## Ein einfaches Model

Nun kannst du ein einfaches Modell mit sklearn definieren. Varianten gibt es viele, es kommt für den Kurs nicht darauf an, welches Modell du wählst und wie gut es ist. Der folgende Lösungsvorschlag ist nur eine Möglichkeit von vielen.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

preprocessor = ColumnTransformer(
    [
        ("numerical", StandardScaler(), numericals),
        ("categorical", OneHotEncoder(sparse_output=False), categoricals),
    ],
)

clf = KNeighborsClassifier(n_neighbors=3)

pipeline = make_pipeline(preprocessor, clf)

----

## Modell evaluieren

Während der Modell-Entwicklung können wir einfach mit der Funtion `cross_val_score` evaluieren.

In [ ]:
from sklearn.model_selection import cross_val_score
n_folds = 10
scores = cross_val_score(pipeline, X_train, y_train, cv=n_folds)
f"{pipeline.steps[-1][-1]} has an accuracy of {scores.mean():0.2f} " +
f"(standard deviation {scores.std():0.2f}) over all {n_folds} folds"

----

## Finale Score prüfen

Am Schlusss prüfen wir unser Model gegen das Test set, um zu die Score zu evaluieren (denn in langen Modellierungsphasen kann es vorkommen, dass wir das Validationset overfitten).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

ConfusionMatrixDisplay.from_estimator(pipeline, X_test, y_test)

print(f"{pipeline.steps[-1][-1]} has an accuracy of {accuracy_score(y_test, y_pred):0.2f}")
plt.show()

----

## Modell mit allen Daten trainieren

Zum Schluss trainieren wir das Modell noch einmal mit allen zur Verfügung stehenden Daten, um auch das letzte Quäntchen Performance rauszuquetschen.

In [ ]:
pipeline.fit(df.drop('class', axis='columns'), df['class'])

----

**Bitte quittiere wiederum auf [Mentimeter](https://www.menti.com/alaxbnek73eu), dass du mit der Übung durch bist**.